# MLFlow Deployment with Explanations

Learn how to implement scoring-time explanations for [MLflow](https://www.mlflow.org/) model. This tutorial produces (1) a new model whose `predict()` method returns both predictions and explanations and (2) a custom deployment of that model to an [online endpoint](https://docs.microsoft.com/azure/machine-learning/concept-endpoints) for that model.

The "explanations" for this model come in the form of local feature importance values. Local feature importance measures the contribution of features for a specific prediction. This tutorial leverages Microsoft's [Responsible AI Toolbox](https://github.com/microsoft/responsible-ai-toolbox) to generate these values, which uses a Mimic explainer, also known as a global surrogate model. You can learn more in the Interpret ML Book's [chapter on global surrogates](https://christophm.github.io/interpretable-ml-book/global.html).

![Example of deployment with explanations](assets/DeploymentExample.png "Example of deployment with explanations")

#### Requirements

- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Learn about workspaces](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace)
- The original model (the one you want predictions for) in your Azure ML workspace - [Learn about models](https://learn.microsoft.com/en-us/azure/machine-learning/tutorial-train-model?view=azureml-api-2)
- The baseline data (the data that will be used to initialize the explainer, this can be the same as the training data for the original model) as a data asset in your Azure ML workspace in MLTable format - [Learn more about data](https://learn.microsoft.com/en-us/azure/machine-learning/concept-data?view=azureml-api-2)

Running this notebook relies on packages in the `requirements.txt` file. Install them or run the command below:

In [ ]:
%pip install -r requirements.txt

## 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

### 1.1 Import required libraries

In [ ]:
import mlflow
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

### 1.2 Configure workspace details and connect **(user input required)**

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial.

Below, enter the information for the Azure ML workspace where the original model and baseline data assets exist. The model with explanations will be created and deployed in this workspace.

In [ ]:
# Enter the details of your AML workspace
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [ ]:
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)
print(f"ML Client: {ml_client}")


# If you are running on a Compute Instance or Compute Cluster in Azure Machine Learning 
# skip the following lines as MLflow is already configured and ready to be used. 
azureml_tracking_uri = ml_client.workspaces.get(
    ml_client.workspace_name
).mlflow_tracking_uri
mlflow.set_tracking_uri(azureml_tracking_uri)
print(f"Tracking URI: {azureml_tracking_uri}")

## 2. Create Wrapper Model

This tutorial uses a wrapper model to provide explanations along with predictions of the original model. The code for this wrapper can be reviewed in `explanation_wrapper.py`.

### 2.1 Model information **(user input required)**

Enter the information about the original model, baseline data, and new model with explanations. The information about the original model and baseline data must match the assets in your Azure ML workspace. the `wrapper_model_name` will be the name of your model with explanations, you may choose any name (e.g. "iris_model_with_explanations").

In [ ]:
# Enter original model information (must match Azure ML asset)
model_name = "<MODEL_NAME>"  # Name of the model for which explanations are desired
model_version = "<MODEL_VERSION>"  # Version of that model

# Enter original model information about features and task
target_column = "<TARGET_COLUMN>"  # The target (aka predicted) column of the data
task_type = "<'classification' | 'regression'>"  # Task type of the model, either 'classification' or 'regression'
categorical_features = []  # Optional, will be calculated from baseline data if None

# Enter baseline data information (must match Azure ML asset)
baseline_data_name = "<BASELINE_DATA_NAME>"  # Name of the baseline data
baseline_data_version = "<BASELINE_DATA_VERSION>"  # Version of that data

# Enter wrapper model name
wrapper_model_name = "<WRAPPER_MODEL_NAME>"  # Name for the model with explanations

In [ ]:
model_uri = f"models:/{model_name}/{model_version}"

### 2.2 Load baseline data

The data used to create the explanation wrapper must match the data used to train the original model. Be sure to drop any columns that were dropped during training. 

In [ ]:
import pandas as pd
import mltable

data_asset = ml_client.data.get(name=baseline_data_name, version=baseline_data_version)
baseline_df = mltable.load(f"azureml:/{data_asset.id}").to_pandas_dataframe()
# Drop any columns that were dropped when training the model. Uncomment and fill in first parameter
# baseline_df = baseline_df.drop([], axis="columns")

print("Baseline Data (first 5 rows):")
baseline_df.head()

### 2.3 Create instance of ExplanationWrapper

In [ ]:
from explanation_wrapper import ExplanationWrapper

explanation_wrapper = ExplanationWrapper(
    model_uri=model_uri,
    baseline_df=baseline_df,
    target_column=target_column,
    task_type=task_type,
    categorical_features=categorical_features,
)

## 3. Register Wrapper Model

### 3.1 Download original model **(user input required)**

Download the original model.

In [ ]:
# Create local folder
import os

local_path = "./artifact/original_model"
if not os.path.exists(local_path):
    os.makedirs(local_path)

In [ ]:
from mlflow.tracking.client import MlflowClient

# Initialize MLFlow client
mlflow_client = MlflowClient()

In [ ]:
# Download run's artifacts/outputs
mlflow.artifacts.download_artifacts(artifact_uri=f"models:/{model_name}/{model_version}", dst_path=local_path)
print("Artifacts downloaded in: {}".format(local_path))
print("Artifacts: {}".format(os.listdir(local_path)))

In [ ]:
mlflow_model_dir = os.path.join(local_path, model_name)

# Show the contents of the MLFlow model folder
os.listdir(mlflow_model_dir)

# You should see a list of files such as the following:
# ['artifacts', 'conda.yaml', 'MLmodel', 'python_env.yaml', 'python_model.pkl', 'requirements.txt']

Update the variable below to be the the file path of the folder that contains the MLFlow model (the `MLmodel` file). To determine this, either (1) use the path provided in the above cell's output or (2) look inside the `artifacts/original_model` folder and then the folder named after the original model. For example, under the following file structure:

![Example downloaded model file structure](assets/ModelFilePathExample.png "Example downloaded model file structure")

You would update `model_file_path` to be './original_model/{model_name}/**model**'.

In [ ]:
model_file_path = (
    f"./artifacts/original_model/{model_name}"
)

### 3.2 Get original model signature

If there is one, load the model signature from Azure ML.

In [ ]:
from mlflow.models import Model

mlflow_model = Model.load(model_uri)
model_signature = mlflow_model.signature
print(f"Model Signature: {model_signature}")

### 3.3 Register wrapper model

Save and then register the model to your workspace. If you are re-running this step, you may need to delete the folder containing the last version of the saved wrapper model.

In [ ]:
artifacts = {"model": model_file_path, "RAI insights": "./artifacts/RAI_Insights"}

In [ ]:
mlflow.pyfunc.save_model(
    path=f"artifacts/{wrapper_model_name}",
    code_path=["./explanation_wrapper.py"],
    conda_env="./env.yml",
    python_model=explanation_wrapper,
    artifacts=artifacts,
    signature=model_signature,
)

In [ ]:
mlflow.register_model(f"file://artifacts/{wrapper_model_name}", wrapper_model_name)

## 4. Deploy Wrapper Model

### 4.1 Deployment Information **(user input required)**

Enter the name of an **existing** endpoint and the deployment name of your choice.

In [ ]:
endpoint_name = "<AML_ENDPOINT_NAME>"
deployment_name = "<DEPLOYMENT_NAME>"

### 4.2 Get wrapper model ID

Verify that the model name and version are correct for the newly registered wrapper model.

In [ ]:
version_list = list(ml_client.models.list(wrapper_model_name))
wrapper_model_version = version_list[0].version
wrapper_model = ml_client.models.get(wrapper_model_name, wrapper_model_version)
print(f"Using model name: {wrapper_model_name}, version: {wrapper_model_version}")

#### 4.3 Create and deploy endpoint

This step may take a while.

In [ ]:
from azure.ai.ml.entities import (
    OnlineRequestSettings,
    ManagedOnlineDeployment,
    ProbeSettings,
)

# Define the deployment
deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=endpoint_name,
    model=wrapper_model.id,
    instance_count=1,
    request_settings=OnlineRequestSettings(request_timeout_ms=90000),
    liveness_probe=ProbeSettings(
        failure_threshold=30,
        success_threshold=1,
        period=100,
        initial_delay=500,
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=30,
        success_threshold=1,
        period=100,
        initial_delay=500,
    ),
)

# Trigger the deployment creation
try:
    ml_client.begin_create_or_update(deployment).wait()
    print("\n---Deployment created successfully---\n")
except Exception as err:
    raise RuntimeError(
        f"Deployment creation failed. Detailed Response:\n{err}"
    ) from err

### 4.4 Assign all traffic to the deployment

Create the traffic configuration:

In [ ]:
import json

traffic_config = {"traffic": {deployment_name: 100}}
traffic_config_path = "artifacts/traffic_config.json"
with open(traffic_config_path, "w") as outfile:
    outfile.write(json.dumps(traffic_config))

Update the configuration:

In [ ]:
from mlflow.deployments import get_deploy_client

deployment_client = get_deploy_client(mlflow.get_tracking_uri())
deployment_client.update_endpoint(
    endpoint=endpoint_name,
    config={"endpoint-config-file": traffic_config_path},
)

## 5. Test Deployment

Testing the deployment can be done through the following steps or the UI in Azure ML portal.

### 5.1 Prepare test data

#### Option 1: Import from csv

Enter the number of samples you plan to use to test the endpoint:

In [ ]:
n = "<NUMBER_OF_SAMPLES>"

Add a csv file with data for your model into this folder and read it in. Again, make sure the column data matches the training data for both the original model and wrapped model.

In [ ]:
sample = (
    pd.read_csv("<CSV_NAME>")
    .sample(n=1)
    .drop(columns=[target_column])
    .reset_index(drop=True)
)

#### Option 2: Write data

Write data directly into the dataframe.

In [ ]:
data = {
    '<COLUMN_1_NAME>': <COLUMN_1_VALUE>,
    '<COLUMN_2_NAME>': <COLUMN_2_VALUE>,
    '<COLUMN_3_NAME>': <COLUMN_3_VALUE>,
    ...
}

In [ ]:
if n == 1:
    sample = pd.DataFrame(data=data, index=[0])
else:
    sample = pd.DataFrame(data=data)
print(f"Sample Data: {sample.head()}")

### 5.2 Invoke the endpoint

Get payload of `predict()` from the endpoint:

In [ ]:
payload = deployment_client.predict(endpoint=endpoint_name, df=sample)

Extract predictions and explanations from payload:

In [ ]:
import numpy as np

if isinstance(payload, pd.DataFrame):
    print("Return type is DataFrame")
    predictions = payload["predictions"].values
    explanations = payload["explanations"].values
elif isinstance(payload, np.ndarray):
    print("Return type is ndarray")
    predictions = payload.item()["predictions"]
    explanations = payload.item()["explanations"]
else:
    print(
        "Return type not supported - either skip the rest of this notebook or write your own code to extract the predictions and explanations lists"
    )

Print and view:

In [ ]:
features = np.array(baseline_df.drop(columns=[target_column]).columns)
if task_type == "classification":
    classes = np.array(baseline_df[target_column].unique())

for i in range(len(predictions)):
    print(f"For data point {i}:")
    print(f"{sample.loc[i]}\n")

    print(f"Prediction: {predictions[i]}\n")

    if task_type == "classification":
        for j in range(len(classes)):
            importances = np.array(explanations[i][j][0])
            explanations_df = pd.DataFrame(
                data={"feature": features, "local importance": importances}
            )
            print(f"Feature importances for class: {classes[j]}")
            if predictions[i] == classes[j]:
                print("This is the predicted class for this row of data")
            else:
                print("This is NOT the predicted class for this row of data")
            print(f"{explanations_df}\n")
    else:
        importances = np.array(explanations[i][0])
        explanations_df = pd.DataFrame(
            data={"feature": features, "local importance": importances}
        )
        print(f"Feature importances: {explanations_df}\n")

    print("\n\n\n")

## 6. Explore data (Optional)

### 6.1 Import plotly

In [ ]:
import plotly.express as px

### 6.2 Feature importances for regression

In [ ]:
if task_type == "regression":
    for i in range(len(explanations)):
        explanations[i] = explanations[i][0]

    features = np.array(baseline_df.drop(columns=[target_column]).columns)
    for i in range(len(predictions)):
        print(f"For data point {i}:")
        print(f"{sample.loc[i]}\n")

        importances = np.array(explanations[i])
        explanations_df = pd.DataFrame(
            data={"feature": features, "local importance": importances}
        )

        plot = px.bar(
            data_frame=explanations_df,
            x="feature",
            y="local importance",
            title="Local Feature Importance",
        )
        plot.show()

        print("\n\n\n")
else:
    print("Task type is not regression - skip this section")

### 6.3 Feature importances for classification

In [ ]:
if task_type == "classification":
    features = np.array(baseline_df.drop(columns=[target_column]).columns)
    classes = np.array(baseline_df[target_column].unique())
    for i in range(len(predictions)):
        print(f"For data point {i}:")
        print(f"{sample.loc[i]}\n")

        aggregated_df = pd.DataFrame(columns=["class", "feature", "local importance"])
        for j in range(len(classes)):
            importances = explanations[i][j][0]
            explanations_df = pd.DataFrame(
                data={"feature": features, "local importance": importances}
            )

            for k in range(len(features)):
                new_row = pd.DataFrame(
                    data=[[classes[j], features[k], explanations[i][j][0][k]]],
                    columns=["class", "feature", "local importance"],
                )
                aggregated_df = pd.concat([aggregated_df, new_row])

            title = f"Local Importance for Class {classes[j]}"
            if predictions[i] == classes[j]:
                title += " (Predicted Class)"
            else:
                title += " (Not the Predicted Class)"
            plot = px.bar(
                data_frame=explanations_df,
                x="feature",
                y="local importance",
                title=title,
            )
            plot.show()

        plot = px.bar(
            data_frame=aggregated_df,
            x="feature",
            y="local importance",
            color="class",
            title="Aggregated Local Importance",
        )
        plot.show()

        print("\n\n\n")
else:
    print("Task type is not classification - skip this section")